In [1]:
from TextNormalizer import TextNormalizer
from flashtext import KeywordProcessor
import re
import pickle

## Загрузка файлов

In [5]:
def save_bin(fname,var):
    with open(fname, 'wb') as fp:
        pickle.dump(var, fp)

In [3]:
def load_bin(fname):
    with open(fname, 'rb') as fp:
        return pickle.load(fp)

In [19]:
def load_txt(fname, vec_delim = '/', phrase_delim='\n'):
    with open(fname, encoding="utf-8") as f:
        return [list(filter(lambda x: x!='',elem.split(phrase_delim))) for elem in f.read().split(vec_delim)]

## Нормализация фраз и документов

In [6]:
def normalize_vecs(vecs):
    return [[' '.join(TextNormalizer.normalize(phrase)) for phrase in vec] for vec in vecs]

In [7]:
def normalize_docs(docs):
    return [' '.join(TextNormalizer.normalize(doc)) for doc in docs]

In [20]:
# список исходных текстов и фраз
docs = load_bin('docs.txt')
vecs = load_txt("vecs.txt")

In [9]:
# делим на предложения тексты и нормализуем их
sdocs = [[' '.join(TextNormalizer.normalize(sent)) for sent in re.split('\. |! |\?',doc)] for doc in docs]

In [10]:
# список групп нормализованных ключевых фраз и список нормализованных текстов
nvecs = load_bin('nvecs')
ndocs = load_bin('ndocs')

## Решение

- делим фразы на слова
- добавляем слова в движок flashtext
- извлекаем слова из прекдложения
- добавляем фразы целиком в движок flashtext
- извлекаем фразы из полученных выше наборов ключевых слов
- возвращаем фразу если она содержится в предложении

In [28]:
def locate_phrase(phrase, clean_phrase, sentence):
    ph_words = phrase.split()
    keyword_processor = KeywordProcessor()
    for word in ph_words:
        keyword_processor.add_keyword(word)
    found_words = ' '.join(keyword_processor.extract_keywords(sentence))
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword(phrase, clean_phrase)
    for elem in found_words:
        found = keyword_processor.extract_keywords(found_words)
        if found:
            return found
    return []

- если вернулась фраза добавляем ее в список
- соответствие текста ключевым фразам = $\frac{|N_{found}|}{|N_{block}|}$
- недостающие фразы - невошедшие в текст (разность множеств)

In [29]:
def relevance(vec, clean_vec, doc):
    found_phrases = []
    for phrase, clean_phrase in zip(vec, clean_vec):
        for sent in doc:
            found = locate_phrase(phrase, clean_phrase, sent)
            if found:
                found_phrases.append(found[0])
                break
    relevance = len(found_phrases)/len(vec)
    missing_phrases = set(clean_vec).difference(set(found_phrases))
    return relevance, missing_phrases

In [30]:
output = relevance(nvecs[0], vecs[0], sdocs[0])

In [41]:
relevance_out = []
missing_phrases_out = []
for idoc, doc in enumerate(sdocs):
    i=1
    for vec, clean_vec in zip(nvecs, vecs):
        rel, mis = relevance(vec, clean_vec, doc)
        #relevance_out.append(rel)
        #missing_phrases_out.append(mis)
        print("\n\nСоответствие ", idoc+1, " документа к ", i, " блоку: ", rel)
        print("Возможно добавить фразы: \n", mis) 
        i+=1



Соответствие  1  документа к  1  блоку:  0.30864197530864196
Возможно добавить фразы: 
 {'системы кондиционирования', 'кондиционер для дома', 'сколько стоит кондиционер', 'кондиционеры воронеж', 'комнатные кондиционеры', 'кондиционеры samsung', 'купить оконный кондиционер', 'купить кондиционер в интернете', 'кондиционеры панасоник', 'кондиционеры для дома', 'кондиционеры отзывы', 'домашние кондиционеры', 'типы кондиционеров', 'сплит системы цена', 'кондиционеры daikin', 'кондиционеры panasonic', 'канальный кондиционер', 'инвертор кондиционер', 'инверторные сплит системы', 'кондиционер киев цена', 'кондиционер сплит', 'кондиционер 12', 'установка кондиционеров в москве недорого', 'купить и установить кондиционер', 'сплит системы в ростове', 'купить инверторный кондиционер', 'кондиционеры сплит системы', 'сплит система купить', 'спрей для кондиционера', 'кондиционеры дешево', 'кондиционеры hitachi', 'купить кондиционер с установкой москва', 'установка сплит систем в краснодаре', 'конди



Соответствие  2  документа к  3  блоку:  0.0
Возможно добавить фразы: 
 {'интернет магазин мобильные кондиционеры', 'мобильный кондиционер недорого', 'бытовые мобильные кондиционеры', 'купить мобильный кондиционер', 'напольный кондиционер купить', 'мобильные кондиционеры напольные', 'кондиционер без воздуховода цена', 'мобильный кондиционер купить', 'переносной кондиционер без воздуховода', 'передвижной кондиционер', 'кондиционер напольный дешево', 'мобильные сплит кондиционеры', 'продам мобильный кондиционер', 'напольный кондиционер москва', 'мобильный кондиционер без воздуховода', 'напольные кондиционеры для дома', 'самый дешевый мобильный кондиционер', 'мобильный кондиционер electrolux', 'мобильные кондиционеры в москве', 'продажа мобильных кондиционеров', 'напольные кондиционеры цены', 'купить переносной кондиционер', 'мобильный кондиционер купить в москве', 'кондиционер без воздуховода купить', 'мобильный кондиционер для дома', 'кондиционер напольный переносной', 'мобильный конд



Соответствие  4  документа к  1  блоку:  0.07407407407407407
Возможно добавить фразы: 
 {'системы кондиционирования', 'сколько стоит кондиционер', 'кондиционер для дома', 'кондиционеры воронеж', 'кондиционеры samsung', 'демонтаж кондиционеров', 'купить оконный кондиционер', 'кондиционеры в москве', 'купить кондиционер в интернете', 'кондиционеры панасоник', 'кондиционеры отзывы', 'кондиционеры для дома', 'домашние кондиционеры', 'типы кондиционеров', 'сплит системы цена', 'кондиционеры daikin', 'кондиционеры panasonic', 'канальный кондиционер', 'инвертор кондиционер', 'инверторные сплит системы', 'кондиционер киев цена', 'кондиционер сплит', 'кондиционер 12', 'недорогие кондиционеры', 'установка кондиционеров в москве недорого', 'купить и установить кондиционер', 'кондиционеры киев', 'заправка кондиционеров', 'магазин кондиционеров', 'обслуживание кондиционеров', 'сплит системы в ростове', 'монтаж кондиционеров', 'установка кондиционеров', 'купить инверторный кондиционер', 'toshiba r

In [31]:
output

(0.30864197530864196,
 {'lg сплит',
  'бытовые кондиционеры',
  'вентилятор',
  'домашние кондиционеры',
  'инвертор кондиционер',
  'инверторные кондиционеры',
  'инверторные сплит системы',
  'инверторный кондиционер',
  'как самому установить кондиционер',
  'канального типа',
  'канальный кондиционер',
  'кассетные кондиционеры',
  'кассетный кондиционер',
  'колонные кондиционеры',
  'комнатные кондиционеры',
  'кондиционер 12',
  'кондиционер samsung',
  'кондиционер акция',
  'кондиционер без воздуховода',
  'кондиционер для дома',
  'кондиционер киев купить',
  'кондиционер киев цена',
  'кондиционер сплит',
  'кондиционер цена',
  'кондиционеры daikin',
  'кондиционеры fujitsu',
  'кондиционеры gree',
  'кондиционеры hitachi',
  'кондиционеры lg',
  'кондиционеры mitsubishi',
  'кондиционеры panasonic',
  'кондиционеры samsung',
  'кондиционеры акция',
  'кондиционеры воронеж',
  'кондиционеры дайкин',
  'кондиционеры дешево',
  'кондиционеры для дома',
  'кондиционеры лджи',


### Tf-idf

In [17]:
nvecs = [TextNormalizer.normalize(text) for text in vecs]

In [18]:
ndocs = [TextNormalizer.normalize(doc) for doc in docs]

In [19]:
dct = Dictionary(nvecs)

In [20]:
vec_corpus = [dct.doc2bow(vec) for vec in nvecs]

In [21]:
doc_corpus = [dct.doc2bow(doc) for doc in ndocs]

In [23]:
model = TfidfModel(vec_corpus,dictionary=dct)

In [24]:
tf_vecs = [model[vec] for vec in vec_corpus]

In [25]:
tf_docs = [model[doc] for doc in doc_corpus]

In [26]:
index = similarities.SparseMatrixSimilarity(tf_vecs, num_features=dct.num_pos)

In [36]:
index[tf_docs[3]]

array([0.07533741, 0.06170711, 0.10711007, 0.78122646], dtype=float32)